In [25]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
#url_list =[url +'&SeasonType=Playoffs' for url in url_list]
url_list =[url +'&SeasonType=Regular+Season'for url in url_list]

def get_tables(url_list):
    data = []
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    for url in url_list:
        
        driver.get(url)
        print(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        
        df= dfs[-1]
        #print(df)
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    driver.close()
    return data
#url_list = [url1]#
def multiyear_shooting(url_list,team_round=0,playoffs = True):
    df_list = []
    start_year = 2023
    for i in range(start_year,2024):
        year = i+1
        season = '&Season='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        if team_round!=0:
            year_url = [url + '&PORound='+str(team_round) for url in year_url]
        frames = get_tables(year_url)

 
        path = str(i+1)+'/playoff_shooting/round'+str(team_round)+'/'
        if playoffs == False:
            path = str(i+1)+'/team_shooting/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['very_tight','tight','open','wide_open']
        
        
        for k in range(len(terms)):
            
            frames[k]['shot_coverage'] = terms[k]
            if playoffs == True:
                frames[k]['round'] = team_round
            frames[k]['year'] = year
        df = pd.concat(frames)
        df.to_csv(path+'team_shooting.csv',index = False)
        df_list.append(df)
    new_df = pd.concat(df_list)
    new_df['TEAMNAME'] = new_df['TEAM']
    df = pd.read_csv('team_shooting.csv')
    df = df[df.year <=start_year]
    names = dict(zip(df.TEAMNAME,df.TEAM))
    names['Los Angeles Clippers'] = 'LAC'
    names['Charlotte Bobcats'] = 'CHA'

    #names
    print(names)
    final_df = pd.concat([df,new_df])
    final_df.replace({'TEAM':names},inplace=True)
    final_df.loc[final_df['TEAMNAME'] =='Los Angeles Clippers', 'TEAM'] = 'LAC'
    final_df.loc[final_df['TEAMNAME'] =='Charlotte Bobcats', 'TEAM'] = 'CHA'

    return final_df

df = multiyear_shooting(url_list,playoffs=False)
print(df)

https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
{'Portland Trail Blazers': 'POR', 'New York Knicks': 'NYK', 'Dallas Mavericks': 'DAL', 'Los Angeles Lakers': 'LAL', 'Denver Nuggets': 'DEN', 'Cleveland Cavaliers': 'CLE', 'Orlando Magic': 'ORL', 'Los Angeles Clippers': 'LAC', 'Washington Wizards': 'WAS', 'Boston Celtics': 'BOS', 'Golden State Warriors': 'GSW', 'Sacramento Kings': 'SAC', 'Minnesota Timberwolves': 'MIN', 'Phoenix Suns': 'PHX', 'Oklahoma Ci

In [27]:
df.to_csv('team_shooting.csv',index = False)

In [5]:
#import pandas as pd
shots = ['wide_open','open','tight','very_tight']
frames = []
for year in range(2014,2025):
    path = str(year)+'/opp_shooting/'
    for shot in shots:
        filepath = path+shot+'.csv'
        df = pd.read_csv(filepath)
        df['shot_coverage'] = shot
        df['year'] = year
        frames.append(df)
opp_master = pd.concat(frames)
opp_master.to_csv('opp_team_shooting.csv',index=False)
frames = []
for year in range(2014,2024):
    path = str(year)+'/playoffs/opp_shooting/'
    for shot in shots:
        filepath = path+shot+'.csv'
        df = pd.read_csv(filepath)
        df['shot_coverage'] = shot
        df['year'] = year
        frames.append(df)
opp_master = pd.concat(frames)
opp_master.to_csv('opp_team_shooting_ps.csv',index=False)
#opp_master

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,year
0,Oklahoma City Thunder,19,19,27.1,167,415,40.2,50.1,12.1,85,185,45.9,15.0,82,230,35.7,wide_open,2014
1,Indiana Pacers,19,19,24.7,148,341,43.4,56.0,8.5,62,118,52.5,16.1,86,223,38.6,wide_open,2014
2,San Antonio Spurs,23,23,16.3,124,294,42.2,55.4,5.5,46,100,46.0,10.8,78,194,40.2,wide_open,2014
3,Miami Heat,20,20,18.5,128,258,49.6,63.6,6.3,56,88,63.6,12.2,72,170,42.4,wide_open,2014
4,Portland Trail Blazers,11,11,20.3,89,188,47.3,58.0,9.3,49,86,57.0,11.0,40,102,39.2,wide_open,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Minnesota Timberwolves,5,5,8.9,19,39,48.7,48.7,8.7,19,38,50.0,0.2,0,1,0.0,very_tight,2023
12,LA Clippers,5,5,6.9,14,30,46.7,46.7,6.5,14,28,50.0,0.5,0,2,0.0,very_tight,2023
13,Cleveland Cavaliers,5,5,6.2,16,26,61.5,61.5,6.2,16,26,61.5,0.0,0,0,-,very_tight,2023
14,Milwaukee Bucks,5,5,5.6,13,25,52.0,54.0,4.7,12,21,57.1,0.9,1,4,25.0,very_tight,2023


In [16]:
old_df = pd.read_csv('team_shooting.csv')
name_dict = dict(zip(old_df.TEAMNAME,old_df.TEAM))
temp = pd.read_csv('opp_team_shooting.csv')
temp['TEAMNAME'] = temp['TEAM']
temp['TEAM'] = temp['TEAMNAME'].map(name_dict)
temp.to_csv('opp_team_shooting.csv',index = False)
temp = pd.read_csv('opp_team_shooting_ps.csv')
temp['TEAMNAME'] = temp['TEAM']
temp['TEAM'] = temp['TEAMNAME'].map(name_dict)
temp.to_csv('opp_team_shooting_ps.csv',index = False)
temp

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,year,TEAMNAME
0,OKC,19,19,27.1,167,415,40.2,50.1,12.1,85,185,45.9,15.0,82,230,35.7,wide_open,2014,Oklahoma City Thunder
1,IND,19,19,24.7,148,341,43.4,56.0,8.5,62,118,52.5,16.1,86,223,38.6,wide_open,2014,Indiana Pacers
2,SAS,23,23,16.3,124,294,42.2,55.4,5.5,46,100,46.0,10.8,78,194,40.2,wide_open,2014,San Antonio Spurs
3,MIA,20,20,18.5,128,258,49.6,63.6,6.3,56,88,63.6,12.2,72,170,42.4,wide_open,2014,Miami Heat
4,POR,11,11,20.3,89,188,47.3,58.0,9.3,49,86,57.0,11.0,40,102,39.2,wide_open,2014,Portland Trail Blazers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,MIN,5,5,8.9,19,39,48.7,48.7,8.7,19,38,50.0,0.2,0,1,0.0,very_tight,2023,Minnesota Timberwolves
636,LAC,5,5,6.9,14,30,46.7,46.7,6.5,14,28,50.0,0.5,0,2,0.0,very_tight,2023,LA Clippers
637,CLE,5,5,6.2,16,26,61.5,61.5,6.2,16,26,61.5,0.0,0,0,-,very_tight,2023,Cleveland Cavaliers
638,MIL,5,5,5.6,13,25,52.0,54.0,4.7,12,21,57.1,0.9,1,4,25.0,very_tight,2023,Milwaukee Bucks


In [18]:
temp.TEAM.unique()

array(['OKC', 'IND', 'SAS', 'MIA', 'POR', 'LAC', 'WAS', 'BKN', 'ATL',
       'HOU', 'GSW', 'DAL', 'CHI', 'TOR', 'MEM', 'CHA', 'CLE', 'MIL',
       'NOP', 'BOS', 'DET', 'UTA', 'PHI', 'MIN', 'DEN', 'ORL', 'LAL',
       'PHX', 'NYK', 'SAC'], dtype=object)